In [1]:
import pandas as pd
from dse_do_utils import DataManager, OptimizationEngine,ScenarioManager
from docplex.mp.model import Model
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

from docplex import __version__
__version__

'2.25.236'

Borrowed from "Ashok D. Belegundu; Tirupathi R. Chandrupatla, Optimization Concepts and Applications in Engineering, 2011 "

VLSI (very large-scale integration) is a process used to build electronic components
such as microprocessors and memory chips comprising millions of transistors.
The first stage of the VLSI design process typically produces a set of
indivisible rectangular blocks called cells. In a second stage, interconnection
information is used to determine the relative placements of these cells. In a
third stage, implementations are selected for the various cells with the goal of
optimizing the total area, which is related to cost of the chips. It is the third
stage, floor plan optimization, for which we give a simple example below.

We are given three rectangular cells. Dimensions of C1 is 5 × 10, C2 can be
chosen as 3 × 8, 2 × 12, or 6 × 4, and C3 can be chosen 5 × 8 or 8 × 5.

 Relative ordering of the cells must satisfy the following vertical and horizontal ordering:

Let ($w_i$ , $h_i$ ), $i = 1, 2, 3$, denote the width and height of cell i, and ($x_i$, $y_i$) denote the coordinates of the left bottom corner of cell $i$. $W$ and $H$ are the sizes of bounding rectangle, where the area supposed to be maximized. 

![](vlsi.png)

$y_1 + h_1 ≤ H$

$y_3 + h_3 ≤ y_2$

$y_2 + h_2 ≤ H$

$x_1 + w_1 ≤ x_2$

$x_1 + w_1 ≤ x_3$

$x_2 + w_2 ≤ W$

$x_3 + w_3 ≤ W$

In [5]:
mdl = Model(name="vlsi")

In [6]:
h1 = mdl.continuous_var(lb = 0, name = 'h1')
h2 = mdl.continuous_var(lb = 0, name = 'h2')
h3 = mdl.continuous_var(lb = 0, name = 'h3')

w1 = mdl.continuous_var(lb = 0, name = 'w1')
w2 = mdl.continuous_var(lb = 0, name = 'w2')
w3 = mdl.continuous_var(lb = 0, name = 'w3')

H = mdl.continuous_var(lb = 0, name = 'H')
W = mdl.continuous_var(lb = 0, name = 'W')

x1 = mdl.continuous_var(lb = 0, name = 'x1')
x2 = mdl.continuous_var(lb = 0, name = 'x2')
x3 = mdl.continuous_var(lb = 0, name = 'x3')

y1 = mdl.continuous_var(lb = 0, name = 'y1')
y2 = mdl.continuous_var(lb = 0, name = 'y2')
y3 = mdl.continuous_var(lb = 0, name = 'y3')



In [7]:
mdl.add_constraint(y1 + h1 <= H)
mdl.add_constraint(y3 + h3 <= y2)
mdl.add_constraint(y2 + h2 <= H)
mdl.add_constraint(x1 + w1 <= x2)
mdl.add_constraint(x1 + w1 <= x3)
mdl.add_constraint(x2 + w2 <= W)
mdl.add_constraint(x3 + w3 <= W)

docplex.mp.LinearConstraint[](w3+x3,LE,W)

Introducing the binary (or 0/1) variables $d_{i j}$ to implement discrete selection

In [8]:
d21 = mdl.binary_var( name = 'd21')
d22 = mdl.binary_var(name = 'd22')
d23 = mdl.binary_var(name = 'd23')

d31 = mdl.binary_var( name = 'd31')
d32 = mdl.binary_var( name = 'd32')

In [9]:
options = [['C1', 5 , 10], ['C2',3,8],['C2',2,12],['C2',6,4],['C3',5,8],['C3',8,5]]
options = pd.DataFrame(options, columns = ['cell','h','w'])
options['cases'] = [0, d21, d22, d23, d31,d32]
options

,cell,h,w,cases
0,C1,5,10,0
1,C2,3,8,d21
2,C2,2,12,d22
3,C2,6,4,d23
4,C3,5,8,d31
5,C3,8,5,d32


In [21]:
mdl.add_constraint(8*d21 + 12 * d22 + 4*d23 == w2)
mdl.add_constraint(3*d21 + 2 * d22 + 6*d23 == h2)

mdl.add_constraint(5*d31 + 8 * d32  == w3)
mdl.add_constraint(8*d31 + 5 * d32  == h3)

mdl.add_constraint(d21 +  d22 + d23  == 1)
mdl.add_constraint(d31 +  d32  == 1)

mdl.add_constraint(d31   == 1)  # added for testing only

docplex.mp.LinearConstraint[](d31,EQ,1)

In [16]:
obj = W * H

In [17]:
mdl.minimize(obj)

mdl.parameters.optimalitytarget = 3
mdl.parameters.dettimelimit = 120

In [18]:
mdl.solve(log_output=True )

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d


CPXPARAM_Read_DataCheck                          1
CPXPARAM_Read_DataCheck                          1


CPXPARAM_OptimalityTarget                        3
CPXPARAM_OptimalityTarget                        3


CPXPARAM_DetTimeLimit                            120
CPXPARAM_DetTimeLimit                            120


Tried aggregator 2 times.
Tried aggregator 2 times.


...

  Real time             =    0.00 sec. (0.00 ticks)
  Real time             =    0.00 sec. (0.00 ticks)


  Sync time (average)   =    0.00 sec.
  Sync time (average)   =    0.00 sec.


  Wait time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.


                          ------------
                          ------------


Total (root+branch&cut) =    0.02 sec. (0.08 ticks)
Total (root+branch&cut) =    0.02 sec. (0.08 ticks)


In [19]:
mdl.print_information()

Model: vlsi
 - number of variables: 19
   - binary=5, integer=0, continuous=14
 - number of constraints: 20
   - linear=20
 - parameters:
     parameters.dettimelimit = 120.00000000000000
     parameters.optimalitytarget = 3
 - objective: minimize quadratic
 - problem type is: MIQP
Model: vlsi
 - number of variables: 19
   - binary=5, integer=0, continuous=14
 - number of constraints: 20
   - linear=20
 - parameters:
     parameters.dettimelimit = 120.00000000000000
     parameters.optimalitytarget = 3
 - objective: minimize quadratic
 - problem type is: MIQP


In [20]:
mdl.print_solution()

DOcplexException: Model<vlsi> did not solve successfully